In [1]:
#Configuration and dataset selection

import sys
import math
import re
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

import spacy
from spacy.language import Language
from spacy.tokens import Doc

import nltk
from nltk.corpus import cmudict

from g2p_en import G2p


# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Paths to the three samples
SAMPLE_PATHS = {
    "small":  "raid_sample_small.csv",
    "medium": "raid_sample_medium.csv",
    "large":  "raid_sample_large.csv",
}

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Set this to one of: "small", "medium", "large"
SELECTED_DATASET = "medium"
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

TEXT_COL = "generation"

df = pd.read_csv(SAMPLE_PATHS[SELECTED_DATASET])
print(f"Loaded {SELECTED_DATASET} dataset with {len(df)} rows.")
df.head(3)


Loaded medium dataset with 12000 rows.


,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,...,digit_ratio,punct_ratio,avg_word_length,std_word_length,entropy_bits,entropy_norm,is_text_like,not_text_reason,n_tokens_ws,length_bin
0,31dac42c-d620-48ef-9f4e-30a072594c4a,cd90a5fb-6aee-4a78-a32d-e34016718deb,0499bbab-4652-479a-8409-95f5ee077c0a,gpt2,sampling,yes,number,poetry,Or From That Sea Of Time,The following is the full text of a poem title...,...,0.000000,0.015625,4.080000,1.610466,4.247145,0.828019,True,NaN,49,short
1,ec269ad5-b494-4269-aff4-9d2b4023db56,5890f7ff-7153-49ba-b4f0-11cb0fc502fa,3a482121-a841-4cc5-b071-058785c10525,mpt-chat,sampling,yes,upper_lower,recipes,Petit Fours,"Write a recipe for ""Petit Fours"".",...,0.012588,0.024169,5.390572,2.193266,4.566332,0.750121,True,NaN,313,long
2,86644bab-0ba0-4b90-bba6-344df0e58ea7,e6d6a10d-6a5b-4801-b160-68ec66db6f9c,e16d6eac-e79b-439d-8f6c-673dcebaecad,cohere-chat,sampling,no,zero_width_space,wiki,Formal specification,"Write the body of a Wikipedia article titled ""...",...,0.002115,0.015865,5.599291,3.701504,4.285282,0.763224,True,NaN,280,long


In [2]:
# Initialize spaCy, CMUdict (NLTK), and g2p_en fallback

# Ensure CMUdict is available
nltk.download('cmudict', quiet=True)
nltk.download('averaged_perceptron_tagger_eng')
CMU = cmudict.dict()  # key: lowercase word, value: list of pronunciations (list of ARPAbet tokens)

# Load spaCy English model (use 'en_core_web_sm' unless you already have md/lg installed)
try:
    nlp: Language = spacy.load("en_core_web_sm")
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

# Ensure sentence boundaries are available (parser usually handles this; add sentencizer if needed)
if "sentencizer" not in nlp.pipe_names and "parser" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")

# g2p_en for OOV coverage
G2P = G2p()

# ARPAbet vowel bases (stress digits removed when checking)
ARPA_VOWELS = {
    "AA", "AE", "AH", "AO", "AW", "AY",
    "EH", "ER", "EY",
    "IH", "IY",
    "OW", "OY",
    "UH", "UW"
}

# Cache syllable counts for speed
_SYLL_CACHE: Dict[str, int] = {}

def cmu_syllables(word: str) -> int | None:
    """
    Returns syllable count using CMUdict if available; else None.
    Policy: use the first pronunciation variant.
    """
    w = word.lower()
    if w not in CMU:
        return None
    phones = CMU[w][0]
    count = 0
    for ph in phones:
        base = re.sub(r"\d", "", ph)
        if base in ARPA_VOWELS:
            count += 1
    return max(count, 1)  # at least one for non-empty alphabetic words

def g2p_syllables(word: str) -> int:
    """
    Returns syllable count using neural g2p_en; counts vowel phonemes.
    """
    w = word.lower()
    if w in _SYLL_CACHE:
        return _SYLL_CACHE[w]
    phones = G2P(w)
    count = 0
    for ph in phones:
        base = re.sub(r"\d", "", ph)
        if base in ARPA_VOWELS:
            count += 1
    # Guard: ensure >=1 for alphabetic tokens
    if count == 0 and re.search(r"[A-Za-z]", w):
        count = 1
    _SYLL_CACHE[w] = count
    return count

def syllables_hybrid(word: str) -> int:
    """
    Hybrid policy: try CMUdict first; if OOV, fall back to g2p_en.
    """
    c = cmu_syllables(word)
    if c is not None:
        return c
    return g2p_syllables(word)


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
# Feature computation utilities using spaCy + CMUdict with g2p_en fallback

def _word_like(tok) -> bool:
    """
    Select lexical tokens (alphabetic, not space).
    spaCy's tok.is_alpha ensures letter-only tokens; change if you want alphanumerics.
    """
    return tok.is_alpha and not tok.is_space

def _alnum_char_count(token_text: str) -> int:
    """Count alphanumeric characters for ARI; excludes whitespace and punctuation."""
    return sum(ch.isalnum() for ch in token_text)

def features_from_doc(doc: Doc) -> Dict[str, float]:
    """
    Computes:
      - avg_word_length
      - type_token_ratio
      - stopword_ratio
      - punctuation_ratio       (punct chars / non-space chars)
      - avg_sentence_length     (words per sentence)
      - sentence_length_std     (std of sentence word counts)
      - flesch_reading_ease
      - gunning_fog
      - smog_index
      - automated_readability_index
    """
    # Sentences
    sents = list(doc.sents) if doc.has_annotation("SENT_START") else [doc]
    n_sents = max(len(sents), 1)

    # Token groups
    word_toks = [t for t in doc if _word_like(t)]
    punct_toks = [t for t in doc if t.is_punct]
    nonspace_toks = [t for t in doc if not t.is_space]

    W = len(word_toks)

    # Characters for ARI and punctuation ratio
    chars_alnum = sum(_alnum_char_count(t.text) for t in nonspace_toks)
    punct_chars = sum(len(t.text) for t in punct_toks)
    nonspace_chars = sum(len(t.text) for t in nonspace_toks)

    # Sentence-level word counts
    sent_word_counts = [sum(1 for t in s if _word_like(t)) for s in sents]
    avg_sentence_length = float(np.mean(sent_word_counts)) if sent_word_counts else 0.0
    sentence_length_std  = float(np.std(sent_word_counts, ddof=0)) if len(sent_word_counts) > 1 else 0.0

    # Word-level lengths
    word_lengths = [len(t.text) for t in word_toks]
    avg_word_length = float(np.mean(word_lengths)) if word_lengths else 0.0

    # Type-token ratio (lowercased forms)
    vocab = {t.text.lower() for t in word_toks}
    type_token_ratio = (len(vocab) / W) if W > 0 else 0.0

    # Stopword ratio via spaCy stop flags
    stop_count = sum(1 for t in word_toks if t.is_stop)
    stopword_ratio = (stop_count / W) if W > 0 else 0.0

    # Punctuation ratio over non-space characters
    punctuation_ratio = (punct_chars / nonspace_chars) if nonspace_chars > 0 else 0.0

    # Syllables (hybrid)
    syll_per_word = [syllables_hybrid(t.text) for t in word_toks] if W > 0 else []
    syll_total = int(np.sum(syll_per_word)) if syll_per_word else 0
    polysyllables = int(np.sum([syl >= 3 for syl in syll_per_word])) if syll_per_word else 0
    complex_words = polysyllables  # standard: >= 3 syllables

    # Rates for readability
    words_per_sentence = (W / n_sents) if n_sents > 0 else 0.0
    syllables_per_word = (syll_total / W) if W > 0 else 0.0
    chars_per_word_ari = (chars_alnum / W) if W > 0 else 0.0

    # Readability indices
    # Flesch Reading Ease
    flesch = 206.835 - 1.015 * words_per_sentence - 84.6 * syllables_per_word

    # Gunning Fog
    fog = 0.4 * (words_per_sentence + 100.0 * (complex_words / W if W > 0 else 0.0))

    # SMOG
    smog = (1.043 * math.sqrt(polysyllables * (30.0 / n_sents)) + 3.1291) if (polysyllables > 0 and n_sents > 0) else 0.0

    # Automated Readability Index
    ari = 4.71 * chars_per_word_ari + 0.5 * words_per_sentence - 21.43

    return {
        "avg_word_length": avg_word_length,
        "type_token_ratio": type_token_ratio,
        "stopword_ratio": stopword_ratio,
        "punctuation_ratio": punctuation_ratio,
        "avg_sentence_length": avg_sentence_length,
        "sentence_length_std": sentence_length_std,
        "flesch_reading_ease": flesch,
        "gunning_fog": fog,
        "smog_index": smog,
        "automated_readability_index": ari,
    }



In [4]:
# N-gram feature extraction utilities

from collections import Counter
from typing import List, Tuple
import math

def extract_ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
    """Extract n-grams from a list of tokens."""
    if len(tokens) < n:
        return []
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]

def ngram_diversity(tokens: List[str], n: int) -> float:
    """
    Calculate n-gram diversity (unique n-grams / total n-grams).
    Returns 0 if no n-grams possible.
    """
    ngrams = extract_ngrams(tokens, n)
    if not ngrams:
        return 0.0
    return len(set(ngrams)) / len(ngrams)

def ngram_entropy(tokens: List[str], n: int) -> float:
    """
    Calculate Shannon entropy of n-gram distribution.
    Returns 0 if no n-grams possible.
    """
    ngrams = extract_ngrams(tokens, n)
    if not ngrams:
        return 0.0
    
    counts = Counter(ngrams)
    total = len(ngrams)
    entropy = 0.0
    
    for count in counts.values():
        prob = count / total
        entropy -= prob * math.log2(prob)
    
    return entropy

def calculate_burstiness(tokens: List[str]) -> float:
    """
    Calculate burstiness coefficient based on word frequency distribution.
    Burstiness = (sigma - mu) / (sigma + mu)
    where mu is mean frequency and sigma is standard deviation.
    Returns 0 if only one unique token.
    """
    if not tokens:
        return 0.0
    
    word_counts = Counter(tokens)
    frequencies = list(word_counts.values())
    
    if len(frequencies) < 2:
        return 0.0
    
    mu = np.mean(frequencies)
    sigma = np.std(frequencies, ddof=0)
    
    if mu + sigma == 0:
        return 0.0
    
    return (sigma - mu) / (sigma + mu)

print("N-gram and burstiness utility functions loaded.")

N-gram and burstiness utility functions loaded.


In [5]:
# Character-level feature extraction

import gzip

def character_ngram_features(text: str, n: int = 3) -> Dict[str, float]:
    """
    Extract character n-gram diversity and entropy.
    Returns diversity ratio and entropy for character n-grams.
    """
    if len(text) < n:
        return 0.0, 0.0
    
    char_ngrams = [text[i:i+n] for i in range(len(text) - n + 1)]
    
    if not char_ngrams:
        return 0.0, 0.0
    
    # Diversity
    diversity = len(set(char_ngrams)) / len(char_ngrams)
    
    # Entropy
    counts = Counter(char_ngrams)
    total = len(char_ngrams)
    entropy = 0.0
    for count in counts.values():
        prob = count / total
        entropy -= prob * math.log2(prob)
    
    return diversity, entropy

def compression_ratio(text: str) -> float:
    """
    Calculate gzip compression ratio: compressed_size / original_size.
    Lower values indicate more compressible (potentially more predictable) text.
    """
    if not text:
        return 1.0
    
    original_bytes = text.encode('utf-8')
    compressed_bytes = gzip.compress(original_bytes)
    
    return len(compressed_bytes) / len(original_bytes)

def character_statistics(text: str) -> Dict[str, float]:
    """
    Extract surface-level character statistics.
    """
    if not text:
        return {
            "uppercase_ratio": 0.0,
            "digit_ratio": 0.0,
            "whitespace_ratio": 0.0,
            "unique_char_count": 0.0,
        }
    
    total_chars = len(text)
    
    return {
        "uppercase_ratio": sum(1 for c in text if c.isupper()) / total_chars,
        "digit_ratio": sum(1 for c in text if c.isdigit()) / total_chars,
        "whitespace_ratio": sum(1 for c in text if c.isspace()) / total_chars,
        "unique_char_count": float(len(set(text))),
    }

print("Character-level feature functions loaded.")

Character-level feature functions loaded.


In [6]:
# Dependency tree and syntactic structure features

def get_tree_depth(token) -> int:
    """
    Recursively calculate the depth of the dependency subtree rooted at token.
    """
    if not list(token.children):
        return 1
    return 1 + max(get_tree_depth(child) for child in token.children)

def dependency_tree_features(doc: Doc) -> Dict[str, float]:
    """
    Extract dependency tree structural features.
    """
    word_toks = [t for t in doc if _word_like(t)]
    
    if not word_toks:
        return {
            "avg_tree_depth": 0.0,
            "max_tree_depth": 0.0,
            "avg_dependency_distance": 0.0,
            "left_dependency_ratio": 0.0,
            "right_dependency_ratio": 0.0,
        }
    
    # Find sentence roots and calculate tree depths
    sents = list(doc.sents) if doc.has_annotation("SENT_START") else [doc]
    tree_depths = []
    
    for sent in sents:
        root = [token for token in sent if token.head == token]
        if root:
            tree_depths.append(get_tree_depth(root[0]))
    
    avg_tree_depth = float(np.mean(tree_depths)) if tree_depths else 0.0
    max_tree_depth = float(np.max(tree_depths)) if tree_depths else 0.0
    
    # Dependency distances (how far apart head and dependent are)
    dep_distances = []
    left_deps = 0
    right_deps = 0
    
    for token in word_toks:
        if token.head != token:  # Not root
            distance = abs(token.i - token.head.i)
            dep_distances.append(distance)
            
            # Track direction
            if token.i < token.head.i:
                left_deps += 1
            else:
                right_deps += 1
    
    avg_dep_distance = float(np.mean(dep_distances)) if dep_distances else 0.0
    total_deps = left_deps + right_deps
    
    left_ratio = left_deps / total_deps if total_deps > 0 else 0.0
    right_ratio = right_deps / total_deps if total_deps > 0 else 0.0
    
    return {
        "avg_tree_depth": avg_tree_depth,
        "max_tree_depth": max_tree_depth,
        "avg_dependency_distance": avg_dep_distance,
        "left_dependency_ratio": left_ratio,
        "right_dependency_ratio": right_ratio,
    }

print("Dependency tree feature functions loaded.")

Dependency tree feature functions loaded.


In [7]:
# Vocabulary sophistication and lexical diversity measures

def hapax_legomena_ratio(tokens: List[str]) -> float:
    """
    Calculate ratio of words appearing exactly once (hapax legomena).
    Indicates vocabulary richness.
    """
    if not tokens:
        return 0.0
    
    word_counts = Counter(tokens)
    hapax_count = sum(1 for count in word_counts.values() if count == 1)
    
    return hapax_count / len(tokens)

def yules_k(tokens: List[str]) -> float:
    """
    Calculate Yule's K measure of lexical diversity.
    More robust to text length than TTR.
    K = 10000 * (M2 - M1) / (M1 * M1)
    where M1 = number of tokens, M2 = sum of (frequency^2 * num_words_with_that_frequency)
    """
    if not tokens:
        return 0.0
    
    word_counts = Counter(tokens)
    M1 = len(tokens)
    
    frequency_spectrum = Counter(word_counts.values())
    M2 = sum(freq * freq * count for freq, count in frequency_spectrum.items())
    
    if M1 <= 1:
        return 0.0
    
    K = 10000 * (M2 - M1) / (M1 * M1)
    return K

def mtld(tokens: List[str], threshold: float = 0.72) -> float:
    """
    Calculate MTLD (Measure of Textual Lexical Diversity).
    Counts how many sequential word segments maintain TTR above threshold.
    """
    if len(tokens) < 10:
        return float(len(set(tokens)))
    
    def compute_factor(token_list):
        ttr = 1.0
        word_set = set()
        factor_count = 0.0
        
        for i, token in enumerate(token_list, 1):
            word_set.add(token)
            ttr = len(word_set) / i
            
            if ttr < threshold:
                factor_count += 1
                word_set = set()
        
        if len(word_set) > 0:
            factor_count += (1.0 - ttr) / (1.0 - threshold)
        
        return factor_count
    
    # Calculate forward and backward
    forward = compute_factor(tokens)
    backward = compute_factor(list(reversed(tokens)))
    
    if forward == 0 and backward == 0:
        return float(len(tokens))
    
    factors = [f for f in [forward, backward] if f > 0]
    mean_factor = np.mean(factors)
    
    return len(tokens) / mean_factor if mean_factor > 0 else float(len(tokens))

def vocabulary_sophistication_features(tokens: List[str]) -> Dict[str, float]:
    """
    Aggregate vocabulary sophistication measures.
    """
    return {
        "hapax_legomena_ratio": hapax_legomena_ratio(tokens),
        "yules_k": yules_k(tokens),
        "mtld": mtld(tokens),
    }

print("Vocabulary sophistication functions loaded.")

Vocabulary sophistication functions loaded.


In [8]:
# Punctuation pattern analysis

def punctuation_patterns(doc: Doc) -> Dict[str, float]:
    """
    Detailed punctuation pattern features beyond simple ratio.
    """
    all_tokens = [t for t in doc if not t.is_space]
    punct_tokens = [t for t in doc if t.is_punct]
    
    if not all_tokens:
        return {
            "comma_ratio": 0.0,
            "period_ratio": 0.0,
            "question_ratio": 0.0,
            "exclamation_ratio": 0.0,
            "semicolon_ratio": 0.0,
            "colon_ratio": 0.0,
            "quote_ratio": 0.0,
        }
    
    total = len(all_tokens)
    
    # Count specific punctuation marks
    punct_text = ''.join([t.text for t in punct_tokens])
    
    return {
        "comma_ratio": punct_text.count(',') / total,
        "period_ratio": punct_text.count('.') / total,
        "question_ratio": punct_text.count('?') / total,
        "exclamation_ratio": punct_text.count('!') / total,
        "semicolon_ratio": punct_text.count(';') / total,
        "colon_ratio": punct_text.count(':') / total,
        "quote_ratio": (punct_text.count('"') + punct_text.count("'")) / total,
    }

print("Punctuation pattern functions loaded.")

Punctuation pattern functions loaded.


In [9]:
from textblob import TextBlob

def sentiment_features(text: str, doc: Doc) -> Dict[str, float]:
    """
    Extract sentiment and emotional tone features.
    Uses TextBlob for polarity and subjectivity.
    """
    if not text or not text.strip():
        return {
            "sentiment_polarity": 0.0,
            "sentiment_subjectivity": 0.0,
            "sentiment_polarity_variance": 0.0,
            "positive_word_ratio": 0.0,
            "negative_word_ratio": 0.0,
            "neutral_sentence_ratio": 0.0,
        }
    
    # Overall document sentiment
    blob = TextBlob(text)
    features = {
        "sentiment_polarity": blob.sentiment.polarity,  # -1 (negative) to 1 (positive)
        "sentiment_subjectivity": blob.sentiment.subjectivity,  # 0 (objective) to 1 (subjective)
    }
    
    # Sentence-level sentiment variance
    sents = list(doc.sents) if doc.has_annotation("SENT_START") else [doc]
    sent_polarities = []
    neutral_count = 0
    
    for sent in sents:
        sent_blob = TextBlob(sent.text)
        polarity = sent_blob.sentiment.polarity
        sent_polarities.append(polarity)
        
        # Count neutral sentences (polarity close to 0)
        if abs(polarity) < 0.1:
            neutral_count += 1
    
    features["sentiment_polarity_variance"] = float(np.var(sent_polarities)) if len(sent_polarities) > 1 else 0.0
    features["neutral_sentence_ratio"] = neutral_count / len(sents) if sents else 0.0
    
    # Positive/negative word ratios using spaCy tokens
    word_toks = [t for t in doc if _word_like(t)]
    if word_toks:
        positive_count = 0
        negative_count = 0
        
        for token in word_toks:
            word_blob = TextBlob(token.text.lower())
            polarity = word_blob.sentiment.polarity
            
            if polarity > 0.1:
                positive_count += 1
            elif polarity < -0.1:
                negative_count += 1
        
        features["positive_word_ratio"] = positive_count / len(word_toks)
        features["negative_word_ratio"] = negative_count / len(word_toks)
    else:
        features["positive_word_ratio"] = 0.0
        features["negative_word_ratio"] = 0.0
    
    return features

In [10]:
# Application NLP.pipe based

BATCH_SIZE = 64
# 1 for deterministic ordering in some environments; -1  all available cores
N_PROCESS = -1

texts = df[TEXT_COL].astype(str).tolist()

feature_rows: List[Dict[str, float]] = []
for text, doc in zip(texts, nlp.pipe(texts, batch_size=BATCH_SIZE, n_process=N_PROCESS)):
    doc_features: Dict[str, float] = {}

    tokens = [t.text for t in doc if _word_like(t)]
    doc_features.update(features_from_doc(doc))

    # Word n-gram diversity/entropy and burstiness
    doc_features["unigram_diversity"] = ngram_diversity(tokens, 1)
    doc_features["bigram_diversity"] = ngram_diversity(tokens, 2)
    doc_features["trigram_diversity"] = ngram_diversity(tokens, 3)
    doc_features["bigram_entropy"] = ngram_entropy(tokens, 2)
    doc_features["trigram_entropy"] = ngram_entropy(tokens, 3)
    doc_features["token_burstiness"] = calculate_burstiness(tokens)

    # Character-level features
    trigram_diversity, trigram_entropy = character_ngram_features(text, n=3)
    doc_features["char_trigram_diversity"] = trigram_diversity
    doc_features["char_trigram_entropy"] = trigram_entropy
    doc_features["compression_ratio"] = compression_ratio(text)
    doc_features.update(character_statistics(text))

    # Syntactic, lexical, punctuation, and sentiment features
    doc_features.update(dependency_tree_features(doc))
    doc_features.update(vocabulary_sophistication_features(tokens))
    doc_features.update(punctuation_patterns(doc))
    doc_features.update(sentiment_features(text, doc))

    feature_rows.append(doc_features)

feat_df = pd.DataFrame(feature_rows)
df_with_features = pd.concat([df.reset_index(drop=True), feat_df.reset_index(drop=True)], axis=1)

print("Computed feature columns:")
print(list(feat_df.columns))
df_with_features.head(3)


Computed feature columns:
['avg_word_length', 'type_token_ratio', 'stopword_ratio', 'punctuation_ratio', 'avg_sentence_length', 'sentence_length_std', 'flesch_reading_ease', 'gunning_fog', 'smog_index', 'automated_readability_index', 'unigram_diversity', 'bigram_diversity', 'trigram_diversity', 'bigram_entropy', 'trigram_entropy', 'token_burstiness', 'char_trigram_diversity', 'char_trigram_entropy', 'compression_ratio', 'uppercase_ratio', 'digit_ratio', 'whitespace_ratio', 'unique_char_count', 'avg_tree_depth', 'max_tree_depth', 'avg_dependency_distance', 'left_dependency_ratio', 'right_dependency_ratio', 'hapax_legomena_ratio', 'yules_k', 'mtld', 'comma_ratio', 'period_ratio', 'question_ratio', 'exclamation_ratio', 'semicolon_ratio', 'colon_ratio', 'quote_ratio', 'sentiment_polarity', 'sentiment_subjectivity', 'sentiment_polarity_variance', 'neutral_sentence_ratio', 'positive_word_ratio', 'negative_word_ratio']


,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,...,exclamation_ratio,semicolon_ratio,colon_ratio,quote_ratio,sentiment_polarity,sentiment_subjectivity,sentiment_polarity_variance,neutral_sentence_ratio,positive_word_ratio,negative_word_ratio
0,31dac42c-d620-48ef-9f4e-30a072594c4a,cd90a5fb-6aee-4a78-a32d-e34016718deb,0499bbab-4652-479a-8409-95f5ee077c0a,gpt2,sampling,yes,number,poetry,Or From That Sea Of Time,The following is the full text of a poem title...,...,0.000000,0.000000,0.018519,0.000000,0.450000,0.650000,0.050625,0.500000,0.040000,0.000000
1,ec269ad5-b494-4269-aff4-9d2b4023db56,5890f7ff-7153-49ba-b4f0-11cb0fc502fa,3a482121-a841-4cc5-b071-058785c10525,mpt-chat,sampling,yes,upper_lower,recipes,Petit Fours,"Write a recipe for ""Petit Fours"".",...,0.011696,0.002924,0.014620,0.002924,0.187625,0.533749,0.035866,0.363636,0.072414,0.024138
2,86644bab-0ba0-4b90-bba6-344df0e58ea7,e6d6a10d-6a5b-4801-b160-68ec66db6f9c,e16d6eac-e79b-439d-8f6c-673dcebaecad,cohere-chat,sampling,no,zero_width_space,wiki,Formal specification,"Write the body of a Wikipedia article titled ""...",...,0.000000,0.000000,0.000000,0.000000,-0.008333,0.287500,0.013217,0.636364,0.021429,0.021429


In [11]:
#  Save enriched dataset and basic descriptive statistics

OUT_WITH_FEATS = f"raid_sample_{SELECTED_DATASET}_with_features_PREPOS.csv"
df_with_features.to_csv(OUT_WITH_FEATS, index=False)
print(f"Saved enriched dataset: {OUT_WITH_FEATS}  (rows: {len(df_with_features)})")

feature_cols = [col for col in df_with_features.columns if col not in df.columns]
print(f"Feature columns saved ({len(feature_cols)} total):")
print(feature_cols)

display(df_with_features[feature_cols].describe(percentiles=[0.1, 0.5, 0.9]).T)


Saved enriched dataset: raid_sample_medium_with_features_PREPOS.csv  (rows: 12000)
Feature columns saved (42 total):
['type_token_ratio', 'stopword_ratio', 'punctuation_ratio', 'avg_sentence_length', 'sentence_length_std', 'flesch_reading_ease', 'gunning_fog', 'smog_index', 'automated_readability_index', 'unigram_diversity', 'bigram_diversity', 'trigram_diversity', 'bigram_entropy', 'trigram_entropy', 'token_burstiness', 'char_trigram_diversity', 'char_trigram_entropy', 'compression_ratio', 'uppercase_ratio', 'whitespace_ratio', 'unique_char_count', 'avg_tree_depth', 'max_tree_depth', 'avg_dependency_distance', 'left_dependency_ratio', 'right_dependency_ratio', 'hapax_legomena_ratio', 'yules_k', 'mtld', 'comma_ratio', 'period_ratio', 'question_ratio', 'exclamation_ratio', 'semicolon_ratio', 'colon_ratio', 'quote_ratio', 'sentiment_polarity', 'sentiment_subjectivity', 'sentiment_polarity_variance', 'neutral_sentence_ratio', 'positive_word_ratio', 'negative_word_ratio']


,count,mean,std,min,10%,50%,90%,max
type_token_ratio,12000.0,0.588571,0.145105,0.006803,0.449778,0.581017,0.752700,1.000000
stopword_ratio,12000.0,0.441644,0.167728,0.000000,0.220585,0.473684,0.609045,1.000000
punctuation_ratio,12000.0,0.029730,0.012610,0.000000,0.016749,0.028137,0.045290,0.229457
avg_sentence_length,12000.0,23.261864,23.031781,0.500000,12.478054,19.777778,30.200000,427.000000
sentence_length_std,12000.0,10.124756,11.268769,0.000000,4.029888,7.878088,16.025289,234.000000
flesch_reading_ease,12000.0,52.543699,31.866539,-345.375000,19.497500,58.332917,81.643773,193.729876
gunning_fog,12000.0,14.574954,9.920579,1.200000,7.747965,13.077524,20.919824,170.800000
smog_index,12000.0,12.194485,4.486948,0.000000,7.447530,11.765961,17.269485,83.107548
automated_readability_index,12000.0,13.035883,12.400888,-5.367672,5.813742,11.199291,18.708750,299.100000
unigram_diversity,12000.0,0.612270,0.147626,0.006803,0.471785,0.605263,0.779412,1.000000


In [12]:
df = df_with_features.copy()

In [13]:
for col in ["avg_sentence_length","flesch_reading_ease","gunning_fog",
            "automated_readability_index","mtld","yules_k","max_tree_depth"]:
    if col in df.columns:
        print(col, "min:", df[col].min(), "max:", df[col].max())


avg_sentence_length min: 0.5 max: 427.0
flesch_reading_ease min: -345.37499999999994 max: 193.72987554112555
gunning_fog min: 1.2000000000000002 max: 170.8
automated_readability_index min: -5.3676718336938265 max: 299.09999999999997
mtld min: 1.0 max: 46381.72000000067
yules_k min: 0.0 max: 9918.69918699187
max_tree_depth min: 2.0 max: 332.0


In [14]:
import numpy as np
import pandas as pd

# -------------------------------
# Helpers
# -------------------------------

NUMERIC_COLS = [
    "avg_sentence_length","sentence_length_std","flesch_reading_ease",
    "gunning_fog","automated_readability_index","mtld","yules_k",
    "bigram_entropy","trigram_entropy","bigram_diversity","trigram_diversity",
    "avg_tree_depth","max_tree_depth","avg_dependency_distance",
    "compression_ratio","uppercase_ratio","unique_char_count","whitespace_ratio"
]

def ensure_numeric(df: pd.DataFrame, cols=NUMERIC_COLS) -> pd.DataFrame:
    """Coerce known feature columns to numeric (if present)."""
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def _q(df, col, q):
    return float(np.nanquantile(df[col].values, q)) if col in df else np.nan

# -------------------------------
# Thresholds and Diagnostics
# -------------------------------

def compute_diagnostic_thresholds(df: pd.DataFrame) -> dict:
    """Data-driven thresholds (quantile-based) + a few hard guards."""
    thr = {}
    thr["asl_hi"]       = max(80.0, _q(df, "avg_sentence_length", 0.99))
    thr["sls_hi"]       = max(80.0, _q(df, "sentence_length_std", 0.99))
    thr["fog_hi"]       = max(60.0, _q(df, "gunning_fog", 0.995))
    thr["ari_hi"]       = max(60.0, _q(df, "automated_readability_index", 0.995))
    thr["fre_lo"]       = min(-50.0, _q(df, "flesch_reading_ease", 0.005))
    thr["mtld_hi"]      = max(500.0, _q(df, "mtld", 0.995))
    thr["yk_hi"]        = max(1000.0, _q(df, "yules_k", 0.995))
    thr["depth_max_hi"] = max(50.0, _q(df, "max_tree_depth", 0.995))
    thr["depth_avg_hi"] = max(25.0, _q(df, "avg_tree_depth", 0.995))
    thr["depdist_hi"]   = max(8.0, _q(df, "avg_dependency_distance", 0.995))
    thr["comp_hi"]      = 1.05
    thr["upper_hi"]     = max(0.15, _q(df, "uppercase_ratio", 0.995))
    thr["uniq_hi"]      = max(80.0, _q(df, "unique_char_count", 0.995))
    thr["ws_lo"]        = min(0.12, _q(df, "whitespace_ratio", 0.005))
    thr["ws_hi"]        = max(0.25, _q(df, "whitespace_ratio", 0.995))
    return thr

def diagnose_feature_outliers(df: pd.DataFrame) -> pd.DataFrame:
    """Return diagnostics dataframe with boolean flags and suspected causes list."""
    # Ensure numeric so comparisons fire correctly
    df = ensure_numeric(df)

    thr = compute_diagnostic_thresholds(df)
    D = pd.DataFrame(index=df.index)

    # Flag definitions
    D["seg_len_extreme"] = (
        (df.get("avg_sentence_length", np.nan) > thr["asl_hi"]) |
        (df.get("sentence_length_std", np.nan) > thr["sls_hi"])
    )
    D["readability_outlier"] = (
        (df.get("flesch_reading_ease", np.nan) < thr["fre_lo"]) |
        (df.get("gunning_fog", np.nan) > thr["fog_hi"]) |
        (df.get("automated_readability_index", np.nan) > thr["ari_hi"])
    )
    D["lexical_instability"] = (
        (df.get("mtld", np.nan) > thr["mtld_hi"]) |
        (df.get("yules_k", np.nan) > thr["yk_hi"])
    )
    D["ngram_edge_effects"] = (
        (df.get("bigram_entropy", np.nan) == 0) |
        (df.get("trigram_entropy", np.nan) == 0) |
        ((df.get("trigram_diversity", np.nan) >= 0.99) & (df.get("trigram_entropy", np.nan) < 1.0)) |
        ((df.get("bigram_diversity", np.nan)   >= 0.99) & (df.get("bigram_entropy", np.nan)   < 1.0))
    )
    D["depth_implausible"] = (
        (df.get("max_tree_depth", np.nan) > thr["depth_max_hi"]) |
        (df.get("avg_tree_depth", np.nan) > thr["depth_avg_hi"])
    )
    D["dep_distance_implausible"] = (df.get("avg_dependency_distance", np.nan) > thr["depdist_hi"])
    D["compression_overhead"] = (df.get("compression_ratio", np.nan) > thr["comp_hi"])
    D["markup_noise"] = (
        ((df.get("uppercase_ratio", np.nan) > thr["upper_hi"]) & (df.get("unique_char_count", np.nan) > thr["uniq_hi"])) |
        (df.get("whitespace_ratio", np.nan) < thr["ws_lo"]) |
        (df.get("whitespace_ratio", np.nan) > thr["ws_hi"])
    )

    # Suspected causes column (pre-allocate + .at)
    D["suspected_causes"] = pd.Series([[] for _ in range(len(D))], index=D.index, dtype=object)
    for i in D.index:
        c = []
        if bool(D.at[i, "depth_implausible"]) or bool(D.at[i, "dep_distance_implausible"]):
            c.append("dependency_depth_computation_bug")
        if bool(D.at[i, "seg_len_extreme"]) and bool(D.at[i, "readability_outlier"]):
            c.append("sentence_segmentation_failure")
        if bool(D.at[i, "lexical_instability"]) or bool(D.at[i, "ngram_edge_effects"]):
            c.append("lexical_metric_instability_or_length_effects")
        if bool(D.at[i, "compression_overhead"]) or bool(D.at[i, "markup_noise"]):
            c.append("markup_or_code_noise")
        D.at[i, "suspected_causes"] = c

    # Severity score
    flag_cols = [c for c in D.columns if c != "suspected_causes"]
    for c in flag_cols:
        D[c] = D[c].fillna(False).astype(bool)
    D["severity"] = D[flag_cols].sum(axis=1).astype(int)

    # Attach thresholds for later inspection
    D.attrs["thresholds"] = thr
    return D

def build_diagnostic_report(diag: pd.DataFrame, top_k: int = 15):
    """Summarize counts by cause and return top offending rows by severity."""
    cause_counts = (
        diag["suspected_causes"].explode().value_counts(dropna=True)
        .rename_axis("cause").to_frame("count")
    )
    flag_counts = (
        diag.drop(columns=["suspected_causes"])
            .sum()
            .sort_values(ascending=False)
            .to_frame("count")
    )
    top_offenders = diag.sort_values("severity", ascending=False).head(top_k)
    return {"cause_counts": cause_counts, "flag_counts": flag_counts, "top_offenders": top_offenders}

# -------------------------------
# Runner / Example usage
# -------------------------------

# Assume you already have `df` with your features
# If your features were just computed and may be strings, the coercion inside
# diagnose_feature_outliers will handle them; coercing here is optional:
# df = ensure_numeric(df)

diag = diagnose_feature_outliers(df)
rep  = build_diagnostic_report(diag, top_k=20)

print("Computed thresholds:")
for k, v in diag.attrs.get("thresholds", {}).items():
    print(f"  {k}: {v}")

from IPython.display import display

print("\nCause counts:")
display(rep["cause_counts"])

print("\nFlag counts:")
display(rep["flag_counts"])

print("\nTop offenders (with original feature values):")
cols_to_show = [
    "avg_sentence_length","sentence_length_std","flesch_reading_ease",
    "gunning_fog","automated_readability_index","mtld","yules_k",
    "avg_tree_depth","max_tree_depth","avg_dependency_distance",
    "compression_ratio","uppercase_ratio","unique_char_count",
    "whitespace_ratio","bigram_entropy","trigram_entropy",
    "bigram_diversity","trigram_diversity"
]
existing_cols = [c for c in cols_to_show if c in df.columns]
display(rep["top_offenders"].join(df[existing_cols], how="left"))





Computed thresholds:
  asl_hi: 119.00500000000011
  sls_hi: 80.0
  fog_hi: 77.73378260869514
  ari_hi: 97.3265883196227
  fre_lo: -115.03797738095234
  mtld_hi: 9027.516866666618
  yk_hi: 1475.1035246435642
  depth_max_hi: 50.0
  depth_avg_hi: 25.0
  depdist_hi: 8.0
  comp_hi: 1.05
  upper_hi: 0.15
  uniq_hi: 80.0
  ws_lo: 0.12
  ws_hi: 0.25

Cause counts:


,count
cause,
lexical_metric_instability_or_length_effects,121
dependency_depth_computation_bug,81
sentence_segmentation_failure,63
markup_or_code_noise,58



Flag counts:


,count
severity,491
seg_len_extreme,164
lexical_instability,120
readability_outlier,65
depth_implausible,53
markup_noise,45
dep_distance_implausible,28
compression_overhead,14
ngram_edge_effects,2



Top offenders (with original feature values):


,seg_len_extreme,readability_outlier,lexical_instability,ngram_edge_effects,depth_implausible,dep_distance_implausible,compression_overhead,markup_noise,suspected_causes,severity,...,max_tree_depth,avg_dependency_distance,compression_ratio,uppercase_ratio,unique_char_count,whitespace_ratio,bigram_entropy,trigram_entropy,bigram_diversity,trigram_diversity
9906,True,True,True,True,True,False,False,True,"[dependency_depth_computation_bug, sentence_se...",6,...,122.0,1.008197,0.031805,0.090976,10.0,0.090237,0.000000,0.000000,0.008197,0.008264
9547,True,True,True,False,True,False,False,True,"[dependency_depth_computation_bug, sentence_se...",5,...,155.0,2.012500,0.093366,0.003276,24.0,0.260442,2.992157,3.503450,0.079167,0.092050
5952,True,True,True,False,False,True,False,True,"[dependency_depth_computation_bug, sentence_se...",5,...,8.0,45.801724,0.067857,0.029390,20.0,0.262246,3.078176,3.392755,0.048851,0.057637
9057,True,True,True,False,True,False,False,True,"[dependency_depth_computation_bug, sentence_se...",5,...,72.0,2.656325,0.035737,0.045484,13.0,0.272255,2.584942,2.584929,0.014320,0.014354
8912,True,True,True,False,False,True,False,True,"[dependency_depth_computation_bug, sentence_se...",5,...,9.0,19.563584,0.070344,0.063949,16.0,0.276579,2.872940,3.127343,0.040462,0.052174
8536,True,True,True,False,True,False,False,False,"[dependency_depth_computation_bug, sentence_se...",4,...,161.0,1.503086,0.059816,0.062117,21.0,0.248466,2.167725,2.192122,0.033951,0.037152
1024,True,True,True,False,False,True,False,False,"[dependency_depth_computation_bug, sentence_se...",4,...,14.0,10.262987,0.542149,0.006203,44.0,0.127378,8.266787,8.262095,1.000000,1.000000
8521,True,True,True,False,False,True,False,False,"[dependency_depth_computation_bug, sentence_se...",4,...,5.0,69.054286,0.056460,0.001526,23.0,0.178026,2.580156,2.623900,0.048571,0.054441
8200,False,True,False,True,False,False,True,True,"[lexical_metric_instability_or_length_effects,...",4,...,2.0,0.000000,1.202247,0.123596,43.0,0.022472,0.000000,0.000000,0.000000,0.000000
4400,True,True,True,False,True,False,False,False,"[dependency_depth_computation_bug, sentence_se...",4,...,140.0,2.789625,0.031739,0.030435,18.0,0.150870,2.321892,2.321904,0.014409,0.014451
